# Import Libraries

In [1]:
import os
import re

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, SimpleRNN, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Load corpus

### Load Positives

In [2]:
train_tweets, train_labels = [], []

pos = os.getcwd() + '/corpus/arabic_tweets/pos/'  # Replace with the actual directory path

# Iterate over each file in the directory
for filename in os.listdir(pos):
    if filename.endswith('.txt'):  # Select only text files
        file_path = os.path.join(pos, filename)
        with open(file_path, 'r', encoding='utf-8-sig') as file:
            file_content = file.read()
            train_tweets.append(file_content)
            train_labels.append("positive")

### Load Negatives

In [3]:
# Get the txt file negative tweet
pos = os.getcwd() + '/corpus/arabic_tweets/neg/'  # Replace with the actual directory path

# Iterate over each file in the directory
for filename in os.listdir(pos):
    if filename.endswith('.txt'):  # Select only text files
        file_path = os.path.join(pos, filename)
        with open(file_path, 'r', encoding='utf-8-sig') as file:
            file_content = file.read()
            train_tweets.append(file_content)
            train_labels.append("negative")

### Build a dataframe

In [4]:
train_dic = {
    'Tweets' : train_tweets,
    'Labels' : train_labels
}

train_corpus = pd.DataFrame(train_dic)
train_corpus.head()

,Tweets,Labels
0,نحن الذين يتحول كل ما نود أن نقوله إلى دعاء لل...,positive
1,وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال...,positive
2,نمش ننوم ما دا ديل ولادنا 💚\n,positive
3,تعدل النت وشفتها ✌\n,positive
4,"🎥 المهمة الأولى في ""جدة"" ✔💪🏼 💙 #الهلال #فيديو_...",positive


# EDA

##### Explore your dataset

In [5]:
train_corpus.tail()

,Tweets,Labels
58746,#أمي فقيدتي وأن مرت الأيام.. وبدأ الجميع بنسيا...,negative
58747,مره في السنه ما كل اسبوع عاد 😢\n,negative
58748,#يوم_الجمعه اسال الله عز وجل في هذا اليوم الفض...,negative
58749,يعني الغاء العقود الاولي كانت تسكيته لنا شسالف...,negative
58750,الفار 🐀 في عهد خليل جلال 😲\n,negative


In [6]:
train_corpus.isnull().sum()

Tweets    0
Labels    0
dtype: int64

In [7]:
train_corpus['Labels'].value_counts()

positive    29849
negative    28902
Name: Labels, dtype: int64

In [8]:
train_corpus.shape

(58751, 2)

In [9]:
train_corpus = train_corpus.drop_duplicates()

In [10]:
train_corpus.shape

(36849, 2)

# Data Preprocessing

### Shuffle all rows

In [11]:
train_corpus = train_corpus.sample(frac=1)

In [12]:
train_corpus

,Tweets,Labels
7298,والله الأيام ذي السرير اللي بجنبه فيش مثل قطعة...,positive
50587,كاتبه بالرغم من الفراق لا أزال اتنفس 🤔 على أسا...,negative
36650,الله أكبر ياحبيبي ياالله مااعظمك وماأعظم شأنك ...,negative
54213,اعطيني اعذارك وانا ابقى بحبك :(\n,negative
7562,#الهلال_رنهم #النصر_نادي_الجاليات #الإتحاد_قده...,positive
...,...,...
31849,يا خاين مالك أمان 😏 اتفقنا نروح مع بعض 🤔😂\n,negative
6634,ياحبي لك .. 💛\n,positive
351,أفضل القلوب : ❤ قلب لايغيب عنه الصدق وأفضل الن...,positive
12867,تجيبك طواري الليل لا راح ثلث الليل و يدعيك قلب...,positive


### Data cleaning

**Hint: remove URLs, Hashtags, alphanumeric characters, punctuation marks, stop words, extra spaces**

In [13]:
URL_pattern = r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
hashtag_pattern = r"#\w+"
mention_pattern = r"@\w+"
alphanumeric_pattern = r"\w*\d\w*"
punctuation_pattern = r"[^\w\s]"
retweet_pattern = r"^RT[\s]+"

In [14]:
def load_stopwords(file_path):
    with open(file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
    return frozenset(stop_set)

def process_text(text, stop_words):
    # Remove URLs
    text = re.sub(URL_pattern, '', text)
    
    # Remove hashtags
    text = re.sub(hashtag_pattern, '', text)
    
    # Remove mention
    text = re.sub(mention_pattern, '', text)

    # Remove alphanumeric characters
    text = re.sub(alphanumeric_pattern, '', text)

    # Remove punctuation marks
    text = re.sub(punctuation_pattern, '', text)
    
    # Remove Retweet marks
    text = re.sub(retweet_pattern, '', text)

    # Remove stop words using the provided set
    text = ' '.join([word for word in text.split() if word.lower() not in stop_words])
    text = ' '.join(text.split())
    return text

#### Now Clean your text using above function or implement it from scrach

In [15]:
stop_words = load_stopwords('C:/Users/dalal/Desktop/RNN_Lab1/corpus/Stop_Words.txt')

In [16]:
print(stop_words)

frozenset({'جميع', 'وان', 'يكون', 'الاولى', 'تم', 'ـ', 'اليوم', 'شخصا', 'صفر', 'مليار', 'ذلك', 'حيث', 'كانت', 'التي', 'بشكل', 'وفي', 'برس', 'الثانية', 'حتى', 'كل', 'مقابل', 'اخرى', 'الف', 'هي', 'اذا', 'غدا', 'وقال', 'امس', 'ثلاثة', 'قال', 'أ', 'الاخيرة', 'وكان', 'نهاية', 'هذه', 'اعلنت', 'غير', 'او', 'اثر', 'صباح', 'الثاني', 'هناك', 'واحد', 'هذا', 'لكن', 'الذين', 'عندما', 'الماضي', 'الذى', 'عاما', 'بسبب', 'بان', 'ضمن', 'مليون', 'عند', 'بها', 'زيارة', 'اما', 'هو', 'عشر', 'نفسه', 'قد', 'فى', 'فيها', 'لم', 'اكد', 'الى', 'بن', 'واوضح', 'ثم', 'واضاف', 'مايو', 'اكثر', 'عن', 'عليها', 'و6', 'وكانت', 'الذي', 'اول', 'اف', 'لوكالة', 'ولا', 'عدد', 'الا', 'الان', 'لن', 'كان', 'وهو', 'السابق', 'المقبل', 'اجل', 'ا', 'واضافت', 'انها', 'ما', 'وقالت', 'لقاء', 'يمكن', 'ضد', 'لدى', 'حاليا', 'كلم', 'بين', 'ايار', 'مساء', 'امام', 'ومن', 'من', 'عشرة', 'منذ', 'بعد', 'احد', 'و', 'ف', 'ولم', 'ان', 'قوة', 'الوقت', 'مع', 'ايام', 'باسم', 'الاول', 'وقد', 'فان', 'على', 'به', 'سنة', 'يوم', 'قبل', 'نحو', 'حوالى', 'منها

In [17]:
train_corpus['Tweets'] = train_corpus['Tweets'].apply(lambda x: process_text(x, stop_words))

In [18]:
train_corpus['Tweets']

7298     والله الأيام ذي السرير اللي بجنبه فيش مثل قطعة...
50587    كاتبه بالرغم الفراق أزال اتنفس أساس صاحبنا أكسجين
36650    الله أكبر ياحبيبي ياالله مااعظمك وماأعظم شأنك ...
54213                         اعطيني اعذارك وانا ابقى بحبك
7562     الهلال دوم كا العاده يحرق قلوب حساده الفن نشوف...
                               ...                        
31849                        يا خاين مالك أمان اتفقنا نروح
6634                                              ياحبي لك
351      أفضل القلوب قلب لايغيب عنه الصدق وأفضل الناس ش...
12867    تجيبك طواري الليل راح ثلث الليل يدعيك قلب تناس...
50216    انتي الزين كله ذوق وحلا ماتحتاجين مدييح لبي قل...
Name: Tweets, Length: 36849, dtype: object

In [19]:
train_corpus['Labels'] = train_corpus['Labels'].map({'positive': 1, 'negative': 0})
train_corpus.head()

,Tweets,Labels
7298,والله الأيام ذي السرير اللي بجنبه فيش مثل قطعة...,1
50587,كاتبه بالرغم الفراق أزال اتنفس أساس صاحبنا أكسجين,0
36650,الله أكبر ياحبيبي ياالله مااعظمك وماأعظم شأنك ...,0
54213,اعطيني اعذارك وانا ابقى بحبك,0
7562,الهلال دوم كا العاده يحرق قلوب حساده الفن نشوف...,1


#### Extra: you could do stemming or lemmatization before training

# Split data to train and test

In [20]:
from sklearn.model_selection import train_test_split

X = train_corpus["Tweets"]
Y = train_corpus["Labels"]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Tokenizer

In [23]:
max_words = 1000 
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

# Text to sequence

In [24]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequence

In [25]:
maxlen = 100  
X_train_padded = pad_sequences(X_train_seq, maxlen=maxlen)
X_test_padded = pad_sequences(X_test_seq, maxlen=maxlen)

# RNN Model

In [27]:
RNN_model = Sequential()
RNN_model.add(Embedding(max_words, 64))
RNN_model.add(SimpleRNN(64))
RNN_model.add(Dense(1, activation='sigmoid'))


RNN_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
RNN_model.fit(X_train_padded, Y_train, epochs=20, batch_size=64, validation_data=(X_test_padded, Y_test))


Epoch 1/20
461/461 ━━━━━━━━━━━━━━━━━━━━ 25s 44ms/step - accuracy: 0.5753 - loss: 0.6717 - val_accuracy: 0.6377 - val_loss: 0.6332
Epoch 2/20
461/461 ━━━━━━━━━━━━━━━━━━━━ 16s 36ms/step - accuracy: 0.6639 - loss: 0.6057 - val_accuracy: 0.6338 - val_loss: 0.6377
Epoch 3/20
461/461 ━━━━━━━━━━━━━━━━━━━━ 18s 39ms/step - accuracy: 0.6667 - loss: 0.5996 - val_accuracy: 0.5811 - val_loss: 0.6582
Epoch 4/20
461/461 ━━━━━━━━━━━━━━━━━━━━ 21s 45ms/step - accuracy: 0.6673 - loss: 0.5957 - val_accuracy: 0.6303 - val_loss: 0.6433
Epoch 5/20
461/461 ━━━━━━━━━━━━━━━━━━━━ 20s 43ms/step - accuracy: 0.6791 - loss: 0.5799 - val_accuracy: 0.6204 - val_loss: 0.6533
Epoch 6/20
461/461 ━━━━━━━━━━━━━━━━━━━━ 20s 44ms/step - accuracy: 0.6891 - loss: 0.5662 - val_accuracy: 0.6282 - val_loss: 0.6503
Epoch 7/20
461/461 ━━━━━━━━━━━━━━━━━━━━ 22s 47ms/step - accuracy: 0.6863 - loss: 0.5665 - val_accuracy: 0.6288 - val_loss: 0.6577
Epoch 8/20
461/461 ━━━━━━━━━━━━━━━━━━━━ 22s 47ms/step - accuracy: 0.6999 - loss: 0.5575 - 

# LSTM Model

In [30]:
LSTM_model = Sequential()
LSTM_model.add(Embedding(max_words, 64))
LSTM_model.add(LSTM(64))
LSTM_model.add(Dense(1, activation='sigmoid'))

LSTM_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
LSTM_model.fit(X_train_padded, Y_train, epochs=20, batch_size=64, validation_data=(X_test_padded, Y_test))


Epoch 1/20
461/461 ━━━━━━━━━━━━━━━━━━━━ 46s 92ms/step - accuracy: 0.5863 - loss: 0.6646 - val_accuracy: 0.6391 - val_loss: 0.6322
Epoch 2/20
461/461 ━━━━━━━━━━━━━━━━━━━━ 51s 112ms/step - accuracy: 0.6618 - loss: 0.6146 - val_accuracy: 0.6380 - val_loss: 0.6329
Epoch 3/20
461/461 ━━━━━━━━━━━━━━━━━━━━ 51s 111ms/step - accuracy: 0.6646 - loss: 0.6038 - val_accuracy: 0.6384 - val_loss: 0.6314
Epoch 4/20
461/461 ━━━━━━━━━━━━━━━━━━━━ 49s 107ms/step - accuracy: 0.6677 - loss: 0.6007 - val_accuracy: 0.6366 - val_loss: 0.6318
Epoch 5/20
461/461 ━━━━━━━━━━━━━━━━━━━━ 48s 104ms/step - accuracy: 0.6730 - loss: 0.5926 - val_accuracy: 0.6335 - val_loss: 0.6340
Epoch 6/20
461/461 ━━━━━━━━━━━━━━━━━━━━ 49s 107ms/step - accuracy: 0.6792 - loss: 0.5830 - val_accuracy: 0.6361 - val_loss: 0.6350
Epoch 7/20
461/461 ━━━━━━━━━━━━━━━━━━━━ 51s 110ms/step - accuracy: 0.6849 - loss: 0.5741 - val_accuracy: 0.6381 - val_loss: 0.6426
Epoch 8/20
461/461 ━━━━━━━━━━━━━━━━━━━━ 50s 108ms/step - accuracy: 0.6918 - loss: 0.

# Evaulation and Comparsion

In [33]:

rnn_loss, rnn_accuracy = RNN_model.evaluate(X_test_padded,Y_test)
print("RNN Accuracy:", rnn_accuracy)

lstm_loss, lstm_accuracy = LSTM_model.evaluate(X_test_padded,Y_test)
print("LSTM Accuracy:", lstm_accuracy)


231/231 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.6169 - loss: 0.7667
RNN Accuracy: 0.6189959049224854
231/231 ━━━━━━━━━━━━━━━━━━━━ 7s 29ms/step - accuracy: 0.6006 - loss: 0.9293
LSTM Accuracy: 0.6093623042106628
